# 쏙쏙Words 단어 점검 행맨 게임
## 대상
- 쏙쏙Words에서 영단어를 암기한 뒤 해당 단어를 점검하고자 하는 학생

## 기대효과
- 이미 외웠던 단어와 문장에서 품사의 중요도에 따라 순차적으로 제시되는 힌트를 보며 외웠던 단어를 어떻게 영작할지에 대한 힌트를 얻을 수 있다고 기대.
- 단어를 정확히 외우는 것을 코사인 유사도를 통해 확인해볼 수 있다. 예를들어 답이 bird인데 birdy라고 하면 그 점수가 다른 오답에 비해 높게 나오는데 이런식으로 정답과 가깝게 맞춰갈 수 있다.

## 플레이
- 실행하게 되면 비어 있는 문장이 전체가 비어있게 나오고 정답만 따로 표시되어있음. 계속 시도를 할 수록 품사의 중요도에 따라 순차적으로 힌트가 제공됨. 총 6번의 기회가 주어지고 정답을 맞추면 게임이 종료되고 기회를 모두 소진 시 원래 답을 제시. 쿄사인 유사도를 함께 제시하며 1.0이 되었을 시 정답으로 표시

In [3]:
# 패키지

import re
import csv
import PyPDF2
import random
from sentence_transformers import SentenceTransformer, util
import spacy
import sys

In [4]:
# PDF 파싱, 빅데이터 9기 신유라님께서 제공


def extract_data_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()

    pattern = re.compile(
        r"([a-zA-Z]+)\s+([가-힣\s,]+)\s+([A-Za-z,.!?\'\s]+)\s+([가-힣,.!?\'\s]+)"
    )
    matches = pattern.findall(text)

    data = []
    for match in matches:
        word = match[0].strip()
        word_meaning = match[1].strip()
        example_en = match[2].strip()
        example_ko = match[3].strip()
        data.append([word, word_meaning, example_en, example_ko])

    return data


def save_data_to_csv(data, output_csv):
    with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["단어", "단어 뜻", "예문", "예문 뜻"])
        writer.writerows(data)


pdf_path = "./T9EE51U70.pdf"

extracted_data = extract_data_from_pdf(pdf_path)

csv_output = "쏙쏙Words.csv"
save_data_to_csv(extracted_data, csv_output)
print(f"CSV 파일이 '{csv_output}'에 저장되었습니다.")

CSV 파일이 '쏙쏙Words.csv'에 저장되었습니다.


---

In [5]:
def load_csv_to_dict(csv_file):
    word_dict = {}

    with open(csv_file, mode="r", encoding="utf-8") as file:
        reader = csv.reader(file)
        next(reader)

        for row in reader:
            word = row[0]
            example_en = row[2]
            word_dict[word] = example_en

    return word_dict


csv_file = "쏙쏙Words.csv"


word_example_dict = load_csv_to_dict(csv_file)


for word, example in word_example_dict.items():
    print(f"단어: {word}, 예문: {example}")

단어: bird, 예문: A bird is in the tree.
단어: idea, 예문: Matt has a good  idea.
단어: wing, 예문: Chickens have wings .
단어: wind, 예문: The wind was strong.
단어: car, 예문: Is this your car?
단어: sky, 예문: Look at the sky.
단어: airplane, 예문: The airplane  is in the sky.
단어: cousin, 예문: My cousins  are kind.
단어: notebook, 예문: Do you have a new notebook ?
단어: vacation, 예문: I like summer vacation .
단어: come, 예문: Can you come here?
단어: museum, 예문: Where is the museum ?
단어: animal, 예문: Kevin loves animals .
단어: horse, 예문: What do horses  eat?
단어: white, 예문: Look at that white rabbit.
단어: clean, 예문: My room is clean now.
단어: break, 예문: ,
단어: building, 예문: The building  is very old.
단어: prize, 예문: The prize was yellow socks.
단어: borrow, 예문: Can I borrow  your pencil?
단어: tomorrow, 예문: Let's meet at two tomorrow .
단어: sleep, 예문: It's time to sleep.
단어: wet, 예문: My shoes are wet.
단어: hello, 예문: Hello, my name is Susan.
단어: early, 예문: I get up early in the morning.
단어: farm, 예문: Are there many cows on the farm?
단

In [8]:
# 게임 코드

quiz = word_example_dict


def hangman_game(word_dict, sentence, correct_word):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    words_in_sentence = sentence.split()

    blank_sentence = [
        "[____]" if word.lower() == correct_word.lower() else "_" * len(word)
        for word in words_in_sentence
    ]

    hint_priority = {
        "AUX": 1,
        "ADJ": 2,
        "DET": 3,
        "NOUN": 4,
        "ADP": 5,
        "ADV": 6,
        "VERB": 7,
        "PRON": 8,
        "PROPN": 9,
        "CCONJ": 10,
        "SCONJ": 11,
        "INTJ": 12,
        "NUM": 13,
        "SYM": 14,
        "PART": 15,
        "PUNCT": 16,
        "X": 17,
    }

    hints = sorted(
        {
            word: pos
            for word, pos in word_dict.items()
            if word.lower() != correct_word.lower()
        }.items(),
        key=lambda item: hint_priority.get(item[1], 100),
    )

    correct_embedding = model.encode([correct_word], convert_to_tensor=True)

    hint_index = 0
    attempts = 6

    print("행맨 게임에 오신 것을 환영합니다!")
    print("문장:", " ".join(blank_sentence))
    sys.stdout.flush()

    while attempts > 0:
        print(f"\n남은 시도 횟수: {attempts}")
        guess = input("정답을 추측해보세요: ").lower()

        if not guess.isalpha():
            print("단어를 입력하세요.")
            continue

        guess_embedding = model.encode([guess], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(guess_embedding, correct_embedding).item()

        if guess == correct_word:
            print(f"축하합니다! 정답은 '{correct_word}'였습니다!")
            print(f"유사도: {similarity:.2f}")
            break
        else:
            print(f"아쉽습니다. '{guess}'는 정답이 아닙니다.")
            print(f"유사도: {similarity:.2f}")
            if similarity > 0.8:
                print("정답에 매우 가까운 단어입니다!")
            elif similarity > 0.5:
                print("꽤 유사한 단어입니다!")
            else:
                print("정답과는 거리가 멉니다.")
            attempts -= 1

            if hint_index < len(hints):
                word_hint, _ = hints[hint_index]
                for i, word in enumerate(words_in_sentence):
                    if word.lower() == word_hint.lower():
                        blank_sentence[i] = word
                print("문장:", " ".join(blank_sentence))
                sys.stdout.flush()
                hint_index += 1

    if attempts == 0:
        print(f"실패했습니다. 정답은 '{correct_word}'였습니다.")
        print(f"유사도: {similarity:.2f}")


nlp = spacy.load("en_core_web_sm")

for word, sentence in quiz.items():
    doc = nlp(sentence)
    word_pos_dict = {token.text: token.pos_ for token in doc}
    hangman_game(word_pos_dict, sentence, word)

c:\Users\user\Desktop\Chunjae_edu\.conda\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


행맨 게임에 오신 것을 환영합니다!
문장: _ [____] __ __ ___ _____

남은 시도 횟수: 6
아쉽습니다. 'where'는 정답이 아닙니다.
유사도: 0.21
정답과는 거리가 멉니다.
문장: _ [____] is __ ___ _____

남은 시도 횟수: 5
아쉽습니다. 'djfh'는 정답이 아닙니다.
유사도: 0.37
정답과는 거리가 멉니다.
문장: A [____] is __ ___ _____

남은 시도 횟수: 4
아쉽습니다. 'sdklfj'는 정답이 아닙니다.
유사도: 0.13
정답과는 거리가 멉니다.
문장: A [____] is __ the _____

남은 시도 횟수: 3
아쉽습니다. 'sldkfj'는 정답이 아닙니다.
유사도: 0.21
정답과는 거리가 멉니다.
문장: A [____] is __ the _____

남은 시도 횟수: 2
아쉽습니다. 'sdlfkj'는 정답이 아닙니다.
유사도: 0.15
정답과는 거리가 멉니다.
문장: A [____] is in the _____

남은 시도 횟수: 1
축하합니다! 정답은 'bird'였습니다!
유사도: 1.00
행맨 게임에 오신 것을 환영합니다!
문장: ____ ___ _ ____ _____

남은 시도 횟수: 6


KeyboardInterrupt: Interrupted by user